In [2]:
from LFOFM import *

wp = WavePlayer()
tempo = wp.setTempo

def mtof(m):
    return 2**((m-69.)/12.) * 440

220.0

In [25]:
bf = (Sin(0.01)+1)*5

f1 = (Sin(freq=0.5, phase=Tri(1.5 * bf))*120 + 640)/2
q1 = 1 + (Sin(0.1)+1)*30
wp.startWaveOSC('f1', '/1/freq', f1)
wp.startWaveOSC('q1', '/1/q', q1)

f2 = (Sin(freq=0.03, phase=Sin(1*bf))*120 + 640)*2
q2 = Sin(0.1)*1 + 2
wp.startWaveOSC('f2', '/2/freq', f2)
wp.startWaveOSC('q2', '/2/q', q2)

f3 = (Sin(phase=Sin(2  * bf))*120 + 640)/3
q3 = Sin(0.1)*3 + 60*Saw(-0.07, phase=Sin(0.06))
wp.startWaveOSC('f3', '/3/freq', f3)
wp.startWaveOSC('q3', '/3/q', q3)

In [98]:
bf = (Sin(0.01)+1)*5

q = 10 + (Sin(freq=Sin(0.08)+2)+1.)/2. * 30

f1 = (Sin(freq=0.5, phase=Tri(1.5 * bf))*120 + 640)/2
q1 = 4 + (Sin(0.2)+1.)/2. * 10
wp.startWaveOSC('f1', '/1/freq', mtof(67))
wp.startWaveOSC('q1', '/1/q', q1)

f2 = (Sin(freq=0.03, phase=Sin(1*bf))*120 + 640)*2
q2 = 4 + (Sin(0.2, 0.7)+1.)/2. * 10
wp.startWaveOSC('f2', '/2/freq', mtof(60))
wp.startWaveOSC('q2', '/2/q', q2)

f3 = (Sin(phase=Sin(2  * bf))*120 + 640)/3
q3 = 4 + (Sin(0.2, 1.4)+1.)/2. * 10
wp.startWaveOSC('f3', '/3/freq', mtof(63))
wp.startWaveOSC('q3', '/3/q', q3)

In [11]:
wp.stopWave('f1')
wp.stopWave('q1')
wp.stopWave('f2')
wp.stopWave('q2')
wp.stopWave('f3')
wp.stopWave('q3')

In [93]:
wp.plotWave(q, 100, 110)